In [26]:
#Import Project Data
# !git clone https://github.com/KodeWil/Airlines-Banks-Sentimen-Analysis-NLP.git

In [27]:
import pandas as pd 
%cd /content/Airlines-Banks-Sentimen-Analysis-NLP/Data
tweetsFrame = pd.read_csv("Tweets.csv")

#Extract the text and the sentiment associated to it
infoFrame = tweetsFrame[["text", "airline_sentiment"]]


/content/Airlines-Banks-Sentimen-Analysis-NLP/Data


In [ ]:
#Replace the airlines name to a standar name "enterprise"
infoFrame["airline_sentiment"] = infoFrame["airline_sentiment"].str.replace("neutral", "1", case = False)
infoFrame["airline_sentiment"] = infoFrame["airline_sentiment"].str.replace("positive", "2", case = False)
infoFrame["airline_sentiment"] = infoFrame["airline_sentiment"].str.replace("negative", "0", case = False)

In [ ]:
companyNames = ["@VirginAmerica", "@united", "@SouthwestAir", "@JetBlue", "@UsAirways", "@AmericanAir"]
#Replace company names for a generic name "company"
for company in companyNames:
  infoFrame["text"] = infoFrame["text"].str.replace(company, "company", case = False)

#Standarized data
infoFrame['airline_sentiment'] = infoFrame['airline_sentiment'].replace({'neutral':'1', 'positive':'2', 'negative': '0'})

In [30]:
#Divide the data in train and test
from sklearn.model_selection import train_test_split
X = infoFrame["text"]
y = infoFrame["airline_sentiment"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=None)
#Encode sentiment type and tokenize sentences
import tensorflow as tf
from keras.layers import Flatten, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding

#Tokenize sentences
oov_tok = "<OOV>" #Define token for missing words
embedding_dim = 50
max_length = 20
trunc_type='post'
padding_type='post'
vocab_size = 20000 #Define the length of words to save
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok) #Set the tokenizer
tokenizer.fit_on_texts(X_train) #Fit it with train set
word_index = tokenizer.word_index #Save the vocabulary
#Tokenize text to sequences (train)
train_sequences = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
#Tokenize text to sequences (test)
test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)



Using TensorFlow backend.


In [ ]:
#Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

In [32]:

import numpy as np
X_train = np.asarray(train_padded, dtype='i4')
y_train = np.asarray(y_train, dtype='i4')
X_test = np.asarray(test_padded, dtype='i4')
y_test = np.asarray(y_test, dtype='i4')

In [33]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)

print("Accuracy:", score)

Accuracy: 0.6338797814207651


In [34]:
#train model
num_epochs = 15
history = model.fit(X_train, y_train, epochs=num_epochs, validation_data=(X_test, y_test), verbose=2)

Epoch 1/15
366/366 - 4s - loss: -5.0135e+00 - accuracy: 0.6062 - val_loss: -3.9631e+01 - val_accuracy: 0.6182
Epoch 2/15
366/366 - 4s - loss: -1.3284e+03 - accuracy: 0.6498 - val_loss: -3.8817e+03 - val_accuracy: 0.6130
Epoch 3/15
366/366 - 4s - loss: -1.9682e+04 - accuracy: 0.6442 - val_loss: -3.6984e+04 - val_accuracy: 0.6301
Epoch 4/15
366/366 - 4s - loss: -1.1186e+05 - accuracy: 0.6232 - val_loss: -1.6021e+05 - val_accuracy: 0.6209
Epoch 5/15
366/366 - 4s - loss: -3.8484e+05 - accuracy: 0.6351 - val_loss: -4.7532e+05 - val_accuracy: 0.6342
Epoch 6/15
366/366 - 4s - loss: -9.9765e+05 - accuracy: 0.6417 - val_loss: -1.0803e+06 - val_accuracy: 0.6216
Epoch 7/15
366/366 - 4s - loss: -2.1344e+06 - accuracy: 0.6180 - val_loss: -2.1394e+06 - val_accuracy: 0.6175
Epoch 8/15
366/366 - 4s - loss: -3.9999e+06 - accuracy: 0.6304 - val_loss: -3.7985e+06 - val_accuracy: 0.6189
Epoch 9/15
366/366 - 4s - loss: -6.8246e+06 - accuracy: 0.6319 - val_loss: -6.2515e+06 - val_accuracy: 0.6202
Epoch 10/1